# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns


In [2]:
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [4]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [5]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [8]:
clickids = set(df[df.action=='click']['id'].unique())
viewids = set(df[df.action=='view']['id'].unique())
df['count']=1
df

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
...,...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view,1
8184,2017-01-18 09:42:12.844575,755912,experiment,view,1
8185,2017-01-18 10:01:09.026482,458115,experiment,view,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1


In [10]:
df.isnull().sum()

timestamp    0
id           0
group        0
action       0
count        0
dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [11]:
#Your code here
control = df[df['group']=='control'].pivot(index='id', columns='action',values='count')
control = control.fillna(value=0)
experiment = df[df['group']=='experiment'].pivot(index='id', columns='action',values='count')
experiment = experiment.fillna(value=0)


In [12]:
#Null hypothesis: there is no difference between the control and experimental homepage
#A;ternative hypothesis: experimental homepage was more effective than control homepage

import scipy.stats as stats
ttest_pvalue = stats.ttest_ind(control, experiment, equal_var=False).pvalue/2
ttest_pvalue

C:\Users\Studies\anaconda3\envs\learn-env\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


array([0.0044664,       nan])

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [15]:
#Your code here
import numpy as np
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate*len(experiment)

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [14]:
#Your code here
n = len(experiment)
p = control_rate
var = n*p*(1-p)
std = np.sqrt(var)
std

24.568547907005815

In [16]:
#Your code here
actual_experiment_clicks = experiment.click.sum()
z_score =(actual_experiment_clicks -expected_experiment_clicks_under_null)/std
z_score 

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [17]:
p_value=stats.norm.sf(z_score)
p_value

0.00012486528006951198

In [18]:
#the pvalue is less than alpha thus we can reject the null hypothesis

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.